# 1. 自动识别

In [1]:
import os

In [2]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '整理好的替代料总表' in file:
        alternate = './input/' + file
    elif '改价表' in file:
        price = './input/' + file
    elif '替代料总表' in file and '整理好的' not in file:
        combination = './input/' + file
    elif 'MRP' in file:
        mrp = './input/' + file
    elif '划分' in file:
        group = './input/' + file
    elif '断货改价' in file:
        last = './input/' + file

print('alternate:     ' + alternate)
print('price:         ' + price)
print('combination:   ' + combination)
print('mrp:           ' + mrp)
print('group:         ' + group)
print('last:          ' + last)

alternate:     ./input/整理好的替代料总表_20240425_202629.xlsx
price:         ./input/改价表 4-23.xlsx
combination:   ./input/替代料总表04-23.xlsx
mrp:           ./input/US MRP20240422-运营.xlsx
group:         ./input/US-eBay 品类划分 2024.2.xlsx
last:          ./input/大账号 断货改价 下午 4.22.xlsx


# 2. 在库在途计算
## 2.1. 替代料计算

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USABS00021-C,USABS00021,,,,,,1
1,USABS00117B-C,USABS00117B,USABS00117,,,,,2
2,USABS00217B-C,USABS00217B,USABS00217,,,,,3
3,USABS00317B-C,USABS00317B,USABS00317,,,,,4
4,USABS00417-C,USABS00417,,,,,,5
...,...,...,...,...,...,...,...,...
48793,USWT80460,USET80460,,,,,,48794
48794,USWT90820,USET90820,,,,,,48795
48795,USWT91020,USET91020,,,,,,48796
48796,USWT91520,USET91520,,,,,,48797


In [5]:
# 替代料表变形
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate.to_excel('./output/1.position.xlsx', index=False)

df_alternate

,主料,SKU,row
0,USABS00021-C,USABS00021-C,1
1,USABS00117B-C,USABS00117B-C,2
2,USABS00217B-C,USABS00217B-C,3
3,USABS00317B-C,USABS00317B-C,4
4,USABS00417-C,USABS00417-C,5
...,...,...,...
106446,USGS6310TNZB-C,USGS6310TNZ,28287
106447,USIC02680L-6-C,USIC02680-6,31054
106448,USIC02680L-C,USIC02680,31055
106449,USIC2A366Z-C,USIC2A366,31162


In [6]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '在途', '在库'],
                         usecols='A, E, F',
                         dtype=str).fillna({'SKU': '',
                                            '在途': 0,
                                            '在库': 0})

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['在库'] = df_price['在库'].astype(int)
df_price['在途'] = df_price['在途'].astype(int)

# 处理后缀
# -N：原单非质量、无理由 等问题的售后退件
# -NC：原单投诉质量问题未核实、货值比较大的订单；原单非质量问题但产品略有小瑕疵（一丢丢脏）
# -D：原单不是质量问题，但退件有缺失一些小部件
df_price['SKU'] = [sku.strip()[:-3] if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip()[:-2] if sku.strip().endswith('-D') else sku.strip() for sku in df_price['SKU'].tolist()]

df_price = df_price.groupby([df_price['SKU']], as_index=False).sum()

df_price

,SKU,在途,在库
0,BCB96454-XH,0,0
1,BPD14040-XH,0,0
2,BPD8550086500+BRS31318,0,0
3,BRR34279-XH,0,0
4,CABM0021A,0,0
...,...,...,...
110324,YHTWS58202-Y-6,0,0
110325,YHTWS5W1ZBA-Y-10,0,0
110326,YHTWS5W1ZBA-Y-8,0,0
110327,YHTWS802AA-Y-6,0,0


In [7]:
print(df_alternate.columns)
print(df_price.columns)

# 找出有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)


df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['SKU', '在途', '在库'], dtype='object')


,主料,SKU,row,在途,在库
0,USABS00021-C,USABS00021-C,1,0.0,10.0
1,USABS00117B-C,USABS00117B-C,2,14.0,1.0
2,USABS00217B-C,USABS00217B-C,3,0.0,0.0
3,USABS00317B-C,USABS00317B-C,4,6.0,39.0
4,USABS00417-C,USABS00417-C,5,8.0,5.0
...,...,...,...,...,...
106446,USGS6310TNZB-C,USGS6310TNZ,28287,0.0,0.0
106447,USIC02680L-6-C,USIC02680-6,31054,0.0,0.0
106448,USIC02680L-C,USIC02680,31055,0.0,0.0
106449,USIC2A366Z-C,USIC2A366,31162,0.0,0.0


In [8]:
# 合并有替代料关系的数据
df_temp = df_temp[['row', '在库', '在途']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

,row,在库,在途
0,1,11.0,0.0
1,2,1.0,14.0
2,3,8.0,0.0
3,4,44.0,6.0
4,5,5.0,8.0
...,...,...,...
48793,48794,6.0,0.0
48794,48795,21.0,11.0
48795,48796,5.0,0.0
48796,48797,2.0,10.0


In [9]:
print(df_alternate.columns)
print(df_temp.columns)

# 更新有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp

Index(['主料', 'SKU', 'row'], dtype='object')
Index(['row', '在库', '在途'], dtype='object')


,主料,SKU,row,在库,在途
0,USABS00021-C,USABS00021-C,1,11.0,0.0
1,USABS00117B-C,USABS00117B-C,2,1.0,14.0
2,USABS00217B-C,USABS00217B-C,3,8.0,0.0
3,USABS00317B-C,USABS00317B-C,4,44.0,6.0
4,USABS00417-C,USABS00417-C,5,5.0,8.0
...,...,...,...,...,...
106446,USGS6310TNZB-C,USGS6310TNZ,28287,23.0,12.0
106447,USIC02680L-6-C,USIC02680-6,31054,62.0,46.0
106448,USIC02680L-C,USIC02680,31055,8.0,4.0
106449,USIC2A366Z-C,USIC2A366,31162,12.0,0.0


In [10]:
print(df_price.columns)
print(df_alternate.columns)

# 找出没有替代料关系的数据
df_temp_2 = pd.merge(left=df_price,
                     right=df_alternate,
                     how='left',
                     left_on='SKU',
                     right_on='SKU').fillna('')

df_temp_2 = df_temp_2[df_temp_2['row'] == '']
df_temp_2['主料'] = df_temp_2['SKU'].tolist()
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途']]

df_temp_2

Index(['SKU', '在途', '在库'], dtype='object')
Index(['主料', 'SKU', 'row'], dtype='object')


,主料,SKU,row,在库,在途
0,BCB96454-XH,BCB96454-XH,,0,0
1,BPD14040-XH,BPD14040-XH,,0,0
2,BPD8550086500+BRS31318,BPD8550086500+BRS31318,,0,0
3,BRR34279-XH,BRR34279-XH,,0,0
4,CABM0021A,CABM0021A,,0,0
...,...,...,...,...,...
110324,YHTWS58202-Y-6,YHTWS58202-Y-6,,0,0
110325,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,,0,0
110326,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,,0,0
110327,YHTWS802AA-Y-6,YHTWS802AA-Y-6,,0,0


In [11]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)
df_price['new_row'] = [i+1 for i in range(len(df_price), len(df_price) * 2)]
df_price['row'] = [row if row != '' else new_row for row, new_row in zip(df_price['row'].tolist(), df_price['new_row'].tolist())]
df_price = df_price[['主料', 'SKU', 'row', '在库', '在途']]

df_price

,主料,SKU,row,在库,在途
0,USABS00021-C,USABS00021-C,1,11.0,0.0
1,USABS00117B-C,USABS00117B-C,2,1.0,14.0
2,USABS00217B-C,USABS00217B-C,3,8.0,0.0
3,USABS00317B-C,USABS00317B-C,4,44.0,6.0
4,USABS00417-C,USABS00417-C,5,5.0,8.0
...,...,...,...,...,...
135477,YHTWS58202-Y-6,YHTWS58202-Y-6,270960,0.0,0.0
135478,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,270961,0.0,0.0
135479,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,270962,0.0,0.0
135480,YHTWS802AA-Y-6,YHTWS802AA-Y-6,270963,0.0,0.0


## 2.2. 组合计算

In [12]:
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype=str)[['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5']].fillna('')

df_combination['数量'] = [amount.strip() for amount in df_combination['数量'].tolist()]
df_combination = df_combination[(df_combination['数量'] != '') & (df_combination['数量'] != '1')].reset_index(drop=True)

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,
3,USCA00103D-2,2,USCA00103D,,,,
4,USCA80215D-2,2,USCA80215D,,,,
...,...,...,...,...,...,...,...
5640,USBRPS19753,1+1,USBPD16530,USBRS54197,,,
5641,USBRPS37107,1+1,USBPD13070,USBRS34371,,,
5642,USBRPS40622,1+1,USBPD13220,USBRS34406,,,
5643,USBRPS49922,1+1,USBPD13220,USBRS34499,,,


In [13]:
print(df_combination.columns)
print(df_price.columns)

df_combination = pd.merge(left=df_combination,
                          right=df_price,
                          how='left',
                          left_on='主料SKU',
                          right_on='SKU').fillna('')

df_combination

Index(['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途'], dtype='object')


,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,在库,在途
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,USWCP52105-2,USWCP52105-2,266487.0,39.0,67.0
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,,USCA51317DLR,USCA51317DLR,13162.0,4.0,10.0
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,,USBCB84615LR,USBCB84615LR,243629.0,8.0,7.0
3,USCA00103D-2,2,USCA00103D,,,,,USCA00103D-2,USCA00103D-2,12387.0,1.0,2.0
4,USCA80215D-2,2,USCA80215D,,,,,USCA80215D-2,USCA80215D-2,13286.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5640,USBRPS19753,1+1,USBPD16530,USBRS54197,,,,,,,,
5641,USBRPS37107,1+1,USBPD13070,USBRS34371,,,,,,,,
5642,USBRPS40622,1+1,USBPD13220,USBRS34406,,,,,,,,
5643,USBRPS49922,1+1,USBPD13220,USBRS34499,,,,,,,,


In [14]:
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU1',
                   right_on='SKU',
                   suffixes=('', '_1')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU2',
                   right_on='SKU',
                   suffixes=('', '_2')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU3',
                   right_on='SKU',
                   suffixes=('', '_3')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU4',
                   right_on='SKU',
                   suffixes=('', '_4')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU5',
                   right_on='SKU',
                   suffixes=('', '_5')).fillna('')

df_combination

,主料SKU,数量,SKU1,SKU2,SKU3,SKU4,SKU5,主料,SKU,row,...,在途_2,SKU_3,在库_3,在途_3,SKU_4,在库_4,在途_4,SKU_5,在库_5,在途_5
0,USWCP52105-2,1+1,USWCP02525,USWCP05210,,,,USWCP52105-2,USWCP52105-2,266487.0,...,,,,,,,,,,
1,USCA51317DLR,1+1,USCA51317D,USCA51318D,,,,USCA51317DLR,USCA51317DLR,13162.0,...,,,,,,,,,,
2,USBCB84615LR,1+1,USBCB84615,USBCB84614,,,,USBCB84615LR,USBCB84615LR,243629.0,...,0.0,,,,,,,,,
3,USCA00103D-2,2,USCA00103D,,,,,USCA00103D-2,USCA00103D-2,12387.0,...,,,,,,,,,,
4,USCA80215D-2,2,USCA80215D,,,,,USCA80215D-2,USCA80215D-2,13286.0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5640,USBRPS19753,1+1,USBPD16530,USBRS54197,,,,,,,...,5.0,,,,,,,,,
5641,USBRPS37107,1+1,USBPD13070,USBRS34371,,,,,,,...,0.0,,,,,,,,,
5642,USBRPS40622,1+1,USBPD13220,USBRS34406,,,,,,,...,0.0,,,,,,,,,
5643,USBRPS49922,1+1,USBPD13220,USBRS34499,,,,,,,...,9.0,,,,,,,,,


In [15]:
from tqdm import tqdm
import math

In [16]:
for i in tqdm(range(len(df_combination)), desc='Progress', ncols=77):
    list_amount = [int(amount.strip()) for amount in df_combination.loc[i, '数量'].split('+')]
    list_stock = []
    list_way = []
    for j in range(len(list_amount)):
        list_stock.append(0 if df_combination.loc[i, '在库_'+str(j+1)] == '' else df_combination.loc[i, '在库_'+str(j+1)] / list_amount[j])
        list_way.append(0 if df_combination.loc[i, '在途_'+str(j+1)] == '' else df_combination.loc[i, '在途_'+str(j+1)] / list_amount[j])
    df_combination.loc[i, '组合在库'] = math.floor(min(list_stock))
    df_combination.loc[i, '组合在途'] = math.floor(min(list_way))
        
df_combination = df_combination[['主料SKU', 'row', '组合在库', '组合在途']]
df_combination = df_combination.sort_values(by=['组合在库', '组合在途'], ascending=[False, False]).reset_index(drop=True)

df_combination

Progress: 100%|████████████████████████| 5645/5645 [00:01<00:00, 4497.85it/s]


,主料SKU,row,组合在库,组合在途
0,USDHO20105LR-C,251887.0,125.0,0.0
1,USCVA61430-2-C,20683.0,123.0,8.0
2,USDHO53436LR-C,251922.0,103.0,0.0
3,USKSAG030LR-C,32697.0,96.0,0.0
4,USDHI15708LR-C,251807.0,96.0,0.0
...,...,...,...,...
5640,USBRPS19753,,0.0,0.0
5641,USBRPS37107,,0.0,0.0
5642,USBRPS40622,,0.0,0.0
5643,USBRPS49922,,0.0,0.0


In [17]:
# 找出有替代料关系的数据
df_temp = df_combination[df_combination['row'] != ''].reset_index(drop=True)
df_temp = df_temp.drop_duplicates(['row'], keep='first', ignore_index=True)

print(df_price.columns)
print(df_combination.columns)

df_temp = pd.merge(left=df_price,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp = df_temp[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

df_temp

Index(['主料', 'SKU', 'row', '在库', '在途'], dtype='object')
Index(['主料SKU', 'row', '组合在库', '组合在途'], dtype='object')


,主料,SKU,row,在库,在途,组合在库,组合在途
0,USABS00021-C,USABS00021-C,1,11.0,0.0,0.0,0.0
1,USABS00117B-C,USABS00117B-C,2,1.0,14.0,0.0,0.0
2,USABS00217B-C,USABS00217B-C,3,8.0,0.0,0.0,0.0
3,USABS00317B-C,USABS00317B-C,4,44.0,6.0,0.0,0.0
4,USABS00417-C,USABS00417-C,5,5.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...
135477,YHTWS58202-Y-6,YHTWS58202-Y-6,270960,0.0,0.0,0.0,0.0
135478,YHTWS5W1ZBA-Y-10,YHTWS5W1ZBA-Y-10,270961,0.0,0.0,0.0,0.0
135479,YHTWS5W1ZBA-Y-8,YHTWS5W1ZBA-Y-8,270962,0.0,0.0,0.0,0.0
135480,YHTWS802AA-Y-6,YHTWS802AA-Y-6,270963,0.0,0.0,0.0,0.0


In [18]:
from datetime import datetime

In [19]:
# 找出没有替代料关系的数据
df_temp_2 = df_combination[df_combination['row'] == ''].reset_index(drop=True)
df_temp_2 = df_temp_2.drop_duplicates(['主料SKU'], keep='first', ignore_index=True)
df_temp_2['row'] = [i+1 for i in range(len(df_price) * 2, len(df_price) * 2 + len(df_temp_2))]
df_temp_2['主料'] = df_temp_2['主料SKU'].tolist()
df_temp_2['SKU'] = df_temp_2['主料SKU'].tolist()
df_temp_2['在库'] = 0
df_temp_2['在途'] = 0
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU', '组合在库', '组合在途']].to_excel('./output/warning-1.组合缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,主料,SKU,row,在库,在途,组合在库,组合在途
0,USABS30280-2-C,USABS30280-2-C,270969,0,0,0.0,0.0
1,USBRPR02307,USBRPR02307,270975,0,0,0.0,0.0
2,USBRPR04307,USBRPR04307,270976,0,0,0.0,0.0
3,USBRPR04707,USBRPR04707,270977,0,0,0.0,0.0
4,USBRPR061813FR,USBRPR061813FR,270978,0,0,0.0,0.0
5,USBRPR10765,USBRPR10765,270967,0,0,1.0,6.0
6,USBRPR13173,USBRPR13173,270979,0,0,0.0,0.0
7,USBRPR16808,USBRPR16808,270980,0,0,0.0,0.0
8,USBRPR37107,USBRPR37107,270981,0,0,0.0,0.0
9,USBRPR652259FR,USBRPR652259FR,270982,0,0,0.0,0.0


In [20]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_price.to_excel('./output/2.stock.xlsx', index=False)

df_price

,主料,SKU,row,在库,在途,组合在库,组合在途
0,USABS00021-C,USABS00021-C,1,11.0,0.0,0.0,0.0
1,USABS00117B-C,USABS00117B-C,2,1.0,14.0,0.0,0.0
2,USABS00217B-C,USABS00217B-C,3,8.0,0.0,0.0,0.0
3,USABS00317B-C,USABS00317B-C,4,44.0,6.0,0.0,0.0
4,USABS00417-C,USABS00417-C,5,5.0,8.0,0.0,0.0
...,...,...,...,...,...,...,...
135510,USCA520T0LR-C,USCA520T0LR-C,270974,0.0,0.0,0.0,0.0
135511,USCA60T70LR-C,USCA60T70LR-C,270972,0.0,0.0,0.0,0.0
135512,USHB12416Y-2-C,USHB12416Y-2-C,270966,0.0,0.0,4.0,0.0
135513,USHBZ41002-2-C,USHBZ41002-2-C,270968,0.0,0.0,0.0,0.0


# 3. MRP整理

In [21]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU', '品类', '层级', '身份'],
                       usecols='B, F, G, I',
                       dtype=str).fillna('')

df_group = pd.read_excel(group,
                         header=0,
                         dtype=str).fillna('')

df_mrp = pd.merge(left=df_mrp,
                  right=df_group,
                  how='left',
                  left_on='品类',
                  right_on='品类').fillna('')

df_mrp = df_mrp[df_mrp['品类负责人'] == 'Hebe'].reset_index(drop=True)

df_mrp

,SKU,品类,层级,身份,品类负责人
0,USCAK14152U-C,CA,TOP1,老品,Hebe
1,USCAK12440-C,CA,TOP10,老品,Hebe
2,USAS0D250-2-C,AS,TOP1,老品,Hebe
3,USHB13214-C,HB,TOP10,老品,Hebe
4,USBCB72928BLR-C,BC,TOP1,老品,Hebe
...,...,...,...,...,...
26970,USWSP8TB01S0-4-YC,WSP,普通,老品,Hebe
26971,USWSP8TB02B0-2-YC,WSP,普通,老品,Hebe
26972,USWSP8TB02B0-4-YC,WSP,普通,老品,Hebe
26973,USWSP8TB03B0-2-YC,WSP,普通,老品,Hebe


In [22]:
print(df_mrp.columns)
print(df_price.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_price,
                  how='left',
                  left_on='SKU',
                  right_on='SKU').fillna('')

df_mrp['row'] = ['' if row == '' else int(row) for row in df_mrp['row'].tolist()]

df_mrp.to_excel('./output/3.mrp.xlsx', index=False)

df_mrp

Index(['SKU', '品类', '层级', '身份', '品类负责人'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,品类,层级,身份,品类负责人,主料,row,在库,在途,组合在库,组合在途
0,USCAK14152U-C,CA,TOP1,老品,Hebe,USCAK14152U-C,16231.0,17.0,62.0,0.0,0.0
1,USCAK12440-C,CA,TOP10,老品,Hebe,USCAK12440-C,15525.0,17.0,163.0,0.0,0.0
2,USAS0D250-2-C,AS,TOP1,老品,Hebe,USAS0D250-2-C,4943.0,18.0,6.0,0.0,0.0
3,USHB13214-C,HB,TOP10,老品,Hebe,USHB13214-C,29605.0,46.0,155.0,0.0,0.0
4,USBCB72928BLR-C,BC,TOP1,老品,Hebe,USBCB72928BLR-C,5568.0,195.0,50.0,14.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
26970,USWSP8TB01S0-4-YC,WSP,普通,老品,Hebe,USWSP8TB01S0-4-YC,268690.0,10.0,0.0,0.0,0.0
26971,USWSP8TB02B0-2-YC,WSP,普通,老品,Hebe,USWSP8TB02B0-2-YC,268691.0,6.0,0.0,0.0,0.0
26972,USWSP8TB02B0-4-YC,WSP,普通,老品,Hebe,USWSP8TB02B0-4-YC,268692.0,13.0,0.0,0.0,0.0
26973,USWSP8TB03B0-2-YC,WSP,普通,老品,Hebe,USWSP8TB03B0-2-YC,268693.0,9.0,0.0,0.0,0.0


In [23]:
df_temp = df_mrp.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,SKU,品类,层级,身份,品类负责人,主料,row,在库,在途,组合在库,组合在途,times_row,times_sku
0,USCAK14152U-C,CA,TOP1,老品,Hebe,USCAK14152U-C,16231.0,17.0,62.0,0.0,0.0,1,1
1,USCAK12440-C,CA,TOP10,老品,Hebe,USCAK12440-C,15525.0,17.0,163.0,0.0,0.0,1,1
2,USAS0D250-2-C,AS,TOP1,老品,Hebe,USAS0D250-2-C,4943.0,18.0,6.0,0.0,0.0,1,1
3,USHB13214-C,HB,TOP10,老品,Hebe,USHB13214-C,29605.0,46.0,155.0,0.0,0.0,1,1
4,USBCB72928BLR-C,BC,TOP1,老品,Hebe,USBCB72928BLR-C,5568.0,195.0,50.0,14.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,USWSP8TB01S0-4-YC,WSP,普通,老品,Hebe,USWSP8TB01S0-4-YC,268690.0,10.0,0.0,0.0,0.0,1,1
26971,USWSP8TB02B0-2-YC,WSP,普通,老品,Hebe,USWSP8TB02B0-2-YC,268691.0,6.0,0.0,0.0,0.0,1,1
26972,USWSP8TB02B0-4-YC,WSP,普通,老品,Hebe,USWSP8TB02B0-4-YC,268692.0,13.0,0.0,0.0,0.0,1,1
26973,USWSP8TB03B0-2-YC,WSP,普通,老品,Hebe,USWSP8TB03B0-2-YC,268693.0,9.0,0.0,0.0,0.0,1,1


In [24]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-1.mrp重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,row,times_row,times_sku


In [25]:
import sys

In [26]:
if len(df_temp_1) != 0:
    sys.exit()

In [27]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-2.mrp缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU
0,USABS00995-C
1,USABS01006-C
2,USABS01006LR-C
3,USABS01975-C
4,USABS01975LR-C
...,...
351,USSTW924638-C
352,USSTW924647-C
353,USSTW925504-C
354,USSTW925507-C


# 4. last与MRP整合

In [29]:
df_last = pd.read_excel(last,
                        header=1,
                        usecols='A, C:D, L:N, P:V, X:Z, AC:AD',
                        dtype=str).fillna('')

df_last['上次序号'] = [int(i.strip()) for i in df_last['序号'].tolist()]
df_last['Date_Left'] = df_last['YKD库存'].tolist()

print(df_last.columns)

df_last

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left'],
      dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,2024-04-17 00:00:02,2024-04-16 00:00:02,2024-04-15 00:00:02,weekly_sale,改价记录,断货时间,总品类调价记录,每日调价记录,上次序号,Date_Left
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,3,5,0,17,,,,,1,195
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,0,0,0,0,,,,,2,18
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,0,1,1,3,,,,,3,20
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,0,3,0,3,,,,,4,73
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,0,0,0,0,,,,,5,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29386,29387,USPSP63G10D-C,,0,0,0,0,0,0,0,0,0,0,0,,,,,29387,0
29387,29388,USSA272971LR,,3,3,3,0,0,0,0,0,0,0,0,,,,,29388,3
29388,29389,USCAK22557LR-C,,4,,,,,,,,,,0,,替代料：USCAK09028-C,,,29389,4
29389,29390,USCA01113-C1,,16,,,,,,,,,,0,,替代料: USCA22557-C,,,29390,16


In [30]:
print(df_last.columns)
print(df_price.columns)

df_last = pd.merge(left=df_last,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_last['row'] = ['' if row == '' else int(row) for row in df_last['row'].tolist()]

df_last

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left'],
      dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,总品类调价记录,每日调价记录,上次序号,Date_Left,主料,row,在库,在途,组合在库,组合在途
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,,,1,195,USBCB72928BLR-C,5568.0,195.0,50.0,14.0,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,,,2,18,USPSP14045D-C,38459.0,22.0,30.0,0.0,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,,,3,20,USBCB75352ZLR-C,5573.0,20.0,51.0,47.0,1.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,,,4,73,USBCB08180ZLR-C,5366.0,73.0,40.0,4.0,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,,,5,17,USPSP01008D-C,38328.0,17.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29386,29387,USPSP63G10D-C,,0,0,0,0,0,0,0,...,,,29387,0,USPSP63G10D-C,38894.0,0.0,0.0,0.0,0.0
29387,29388,USSA272971LR,,3,3,3,0,0,0,0,...,,,29388,3,USSA272971LR,263007.0,3.0,0.0,0.0,0.0
29388,29389,USCAK22557LR-C,,4,,,,,,,...,,,29389,4,USCAK09028-C,13894.0,4.0,10.0,16.0,0.0
29389,29390,USCA01113-C1,,16,,,,,,,...,,,29390,16,USCA22557-C,12867.0,16.0,2.0,0.0,0.0


In [31]:
df_temp = df_last.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,上次序号,Date_Left,主料,row,在库,在途,组合在库,组合在途,times_row,times_sku
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,1,195,USBCB72928BLR-C,5568.0,195.0,50.0,14.0,0.0,1,1
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,2,18,USPSP14045D-C,38459.0,22.0,30.0,0.0,0.0,1,1
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,3,20,USBCB75352ZLR-C,5573.0,20.0,51.0,47.0,1.0,1,1
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,4,73,USBCB08180ZLR-C,5366.0,73.0,40.0,4.0,0.0,1,1
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,5,17,USPSP01008D-C,38328.0,17.0,0.0,0.0,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29386,29387,USPSP63G10D-C,,0,0,0,0,0,0,0,...,29387,0,USPSP63G10D-C,38894.0,0.0,0.0,0.0,0.0,1,1
29387,29388,USSA272971LR,,3,3,3,0,0,0,0,...,29388,3,USSA272971LR,263007.0,3.0,0.0,0.0,0.0,1,1
29388,29389,USCAK22557LR-C,,4,,,,,,,...,29389,4,USCAK09028-C,13894.0,4.0,10.0,16.0,0.0,1,1
29389,29390,USCA01113-C1,,16,,,,,,,...,29390,16,USCA22557-C,12867.0,16.0,2.0,0.0,0.0,1,1


In [32]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-2.last重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

,SKU,row,times_row,times_sku


In [33]:
if len(df_temp_1) != 0:
    sys.exit()

In [34]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-3.last缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU
0,USAS57501
1,USAS65602
2,USAS94613
3,USAST49357-C1
4,USBMC39628-C
...,...
194,USWHS10574
195,USWHS23735
196,USWHS43222
197,USWHS81153


In [35]:
print(df_last.columns)
print(df_mrp.columns)

# 找出有库存信息的数据
df_temp = pd.merge(left=df_last[df_last['row'] != ''],
                   right=df_mrp,
                   how='left',
                   left_on='row',
                   right_on='row',
                   suffixes=('', '_mrp')).fillna('')

df_temp['SKU'] = [sku if sku_mrp == '' else sku_mrp for sku, sku_mrp in zip(df_temp['SKU'].tolist(), df_temp['SKU_mrp'].tolist())]
df_temp['Note'] = ['' if charge == 'Hebe' else 'last only, not in MRP' for charge in df_temp['品类负责人'].tolist()]
list_column = df_temp.columns.tolist()
list_column.remove('SKU_mrp')
list_column.remove('主料_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp = df_temp[list_column]

df_temp

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',                '主料',
                     'row',                '在库',                '在途',
                    '组合在库',              '组合在途'],
      dtype='object')
Index(['SKU', '品类', '层级', '身份', '品类负责人', '主料', 'row', '在库', '在途', '组合在库',
       '组合在途'],
      dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,row,在库,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,5568.0,195.0,50.0,14.0,0.0,BC,TOP1,老品,Hebe,
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,38459.0,22.0,30.0,0.0,0.0,PSP,TOP10,老品,Hebe,
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,5573.0,20.0,51.0,47.0,1.0,BC,TOP1,老品,Hebe,
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,5366.0,73.0,40.0,4.0,0.0,BC,TOP1,老品,Hebe,
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,38328.0,17.0,0.0,0.0,0.0,PSP,普通,老品,Hebe,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29187,29387,USPSP63G10D-C,,0,0,0,0,0,0,0,...,38894.0,0.0,0.0,0.0,0.0,PSP,普通,清仓,Hebe,
29188,29388,USSA272971LR,,3,3,3,0,0,0,0,...,263007.0,3.0,0.0,0.0,0.0,SA,普通,老品,Hebe,
29189,29389,USCAK22557LR-C,,4,,,,,,,...,13894.0,4.0,10.0,16.0,0.0,CA,普通,断货,Hebe,
29190,29390,USCA22557-C,,16,,,,,,,...,12867.0,16.0,2.0,0.0,0.0,CA,普通,老品,Hebe,


In [36]:
print(df_last.columns)
print(df_mrp.columns)

# 找出没有库存信息的数据
df_temp_2 = pd.merge(left=df_last[df_last['row'] == ''],
                     right=df_mrp,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_mrp')).fillna('')

df_temp_2['在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['在库'].tolist(), df_temp_2['在库_mrp'].tolist())]
df_temp_2['在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['在途'].tolist(), df_temp_2['在途_mrp'].tolist())]
df_temp_2['组合在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['组合在库'].tolist(), df_temp_2['组合在库_mrp'].tolist())]
df_temp_2['组合在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['组合在途'].tolist(), df_temp_2['组合在途_mrp'].tolist())]
df_temp_2['Note'] = ['' if charge == 'Hebe' else 'last only, not in MRP' for charge in df_temp_2['品类负责人'].tolist()]
list_column = df_temp_2.columns.tolist()
list_column.remove('主料_mrp')
list_column.remove('row_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',                '主料',
                     'row',                '在库',                '在途',
                    '组合在库',              '组合在途'],
      dtype='object')
Index(['SKU', '品类', '层级', '身份', '品类负责人', '主料', 'row', '在库', '在途', '组合在库',
       '组合在途'],
      dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,row,在库,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note
0,512,USAS57501,0,,,,0,0,0,0,...,,,,,,AS,普通,清仓,Hebe,
1,514,USAS65602,0,,,,0,0,0,0,...,,,,,,AS,普通,清仓,Hebe,
2,515,USAS94613,0,,,,0,0,0,0,...,,,,,,AS,普通,清仓,Hebe,
3,4583,USHB12479-2-C,USHB12479-2,,,,0,0,0,0,...,,,,,,,,,,"last only, not in MRP"
4,6926,USWHS10571-C,0,,,,0,0,0,0,...,,,,,,,,,,"last only, not in MRP"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,29298,USCA50S90-C,,,,,0,0,0,0,...,,,,,,CA,普通,新品,Hebe,
195,29299,USCA51360-C,,,,,0,0,0,0,...,,,,,,CA,TOP10,新品,Hebe,
196,29300,USCA52351-C,,,,,0,0,0,0,...,,,,,,CA,普通,新品,Hebe,
197,29304,USCA7Z550-C,,,,,0,0,0,0,...,,,,,,CA,TOP20,新品,Hebe,


In [37]:
# 整合数据
df_temp = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_temp

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,row,在库,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,5568.0,195.0,50.0,14.0,0.0,BC,TOP1,老品,Hebe,
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,38459.0,22.0,30.0,0.0,0.0,PSP,TOP10,老品,Hebe,
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,5573.0,20.0,51.0,47.0,1.0,BC,TOP1,老品,Hebe,
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,5366.0,73.0,40.0,4.0,0.0,BC,TOP1,老品,Hebe,
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,38328.0,17.0,0.0,0.0,0.0,PSP,普通,老品,Hebe,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29386,29298,USCA50S90-C,,,,,0,0,0,0,...,,,,,,CA,普通,新品,Hebe,
29387,29299,USCA51360-C,,,,,0,0,0,0,...,,,,,,CA,TOP10,新品,Hebe,
29388,29300,USCA52351-C,,,,,0,0,0,0,...,,,,,,CA,普通,新品,Hebe,
29389,29304,USCA7Z550-C,,,,,0,0,0,0,...,,,,,,CA,TOP20,新品,Hebe,


In [38]:
print(df_mrp.columns)
print(df_last.columns)

df_temp_2 = pd.merge(left=df_mrp,
                     right=df_last,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_last')).fillna('')

df_temp_2 = df_temp_2[df_temp_2['序号'] == ''].reset_index(drop=True)
for i in range(len(df_temp_2)):
    if df_temp_2.loc[i, 'SKU'] not in df_temp['SKU'].tolist():
        df_temp_2.loc[i, 'Note'] = 'MRP only, not in last'

if len(df_temp_2) > 0:
    df_temp_2 = df_temp_2[df_temp_2['Note'] == 'MRP only, not in last'].reset_index(drop=True)

list_column = df_temp_2.columns.tolist()
list_column.remove('主料_last')
list_column.remove('row_last')
list_column.remove('在库_last')
list_column.remove('在途_last')
list_column.remove('组合在库_last')
list_column.remove('组合在途_last')
df_temp_2 = df_temp_2[list_column]

df_temp_2

Index(['SKU', '品类', '层级', '身份', '品类负责人', '主料', 'row', '在库', '在途', '组合在库',
       '组合在途'],
      dtype='object')
Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',                '主料',
                     'row',                '在库',                '在途',
                    '组合在库',              '组合在途'],
      dtype='object')


,SKU,品类,层级,身份,品类负责人,主料,row,在库,在途,组合在库,...,2024-04-16 00:00:02,2024-04-15 00:00:02,weekly_sale,改价记录,断货时间,总品类调价记录,每日调价记录,上次序号,Date_Left,Note
0,USABS00995-C,ABS,普通,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
1,USABS01006-C,ABS,普通,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
2,USABS01006LR-C,ABS,普通,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
3,USABS01975-C,ABS,普通,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
4,USABS01975LR-C,ABS,普通,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,USSTW924638-C,STW,TOP10,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
218,USSTW924647-C,STW,TOP10,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
219,USSTW925504-C,STW,TOP10,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"
220,USSTW925507-C,STW,TOP10,新品,Hebe,,,,,,...,,,,,,,,,,"MRP only, not in last"


In [39]:
df = pd.concat([df_temp, df_temp_2], ignore_index=True)

df

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,row,在库,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,5568.0,195.0,50.0,14.0,0.0,BC,TOP1,老品,Hebe,
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,38459.0,22.0,30.0,0.0,0.0,PSP,TOP10,老品,Hebe,
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,5573.0,20.0,51.0,47.0,1.0,BC,TOP1,老品,Hebe,
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,5366.0,73.0,40.0,4.0,0.0,BC,TOP1,老品,Hebe,
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,38328.0,17.0,0.0,0.0,0.0,PSP,普通,老品,Hebe,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last"
29609,,USSTW924647-C,,,,,,,,,...,,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last"
29610,,USSTW925504-C,,,,,,,,,...,,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last"
29611,,USSTW925507-C,,,,,,,,,...,,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last"


# 5. 其他计算

In [40]:
df_missing = pd.read_excel(price,
                           sheet_name=1,
                           header=0,
                           names=['SKU', '数量'],
                           usecols='D, F',
                           dtype=str).fillna('')

df_missing = df_missing[df_missing['SKU'] != ''].reset_index(drop=True)

df_missing['数量'] = df_missing['数量'].astype(int)
df_missing = df_missing.groupby([df_missing['SKU']], as_index=False).sum()

df_missing

,SKU,数量
0,USABS10461-C,2
1,USABS11330-C,1
2,USABS12530-C,1
3,USABS13140-C,1
4,USABS1420-C,1
...,...,...
1631,USWSP4TH03B0-4-YC,1
1632,USWSP54507B1-4-YC,1
1633,USWSP54530B1-4-YC,1
1634,USWSP65507B1-4-YC,1


In [41]:
print(df_missing.columns)
print(df_price.columns)

df_missing = pd.merge(left=df_missing,
                      right=df_price,
                      how='left',
                      left_on='SKU',
                      right_on='SKU').fillna('')

df_missing

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,数量,主料,row,在库,在途,组合在库,组合在途
0,USABS10461-C,2,USABS10461-C,59.0,5.0,3.0,0.0,0.0
1,USABS11330-C,1,USABS11330-C,92.0,29.0,17.0,0.0,0.0
2,USABS12530-C,1,USABS12530-C,128.0,12.0,0.0,0.0,0.0
3,USABS13140-C,1,USABS13140-C,145.0,1.0,4.0,0.0,0.0
4,USABS1420-C,1,USABS1420-C,176.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1631,USWSP4TH03B0-4-YC,1,USWSP4TH03B0-4-YC,268278.0,22.0,0.0,0.0,0.0
1632,USWSP54507B1-4-YC,1,USWSP54507B1-4-YC,268312.0,6.0,0.0,0.0,0.0
1633,USWSP54530B1-4-YC,1,USWSP54530B1-4-YC,268368.0,2.0,0.0,0.0,0.0
1634,USWSP65507B1-4-YC,1,USWSP65507B1-4-YC,268602.0,30.0,0.0,0.0,0.0


In [42]:
df_temp = df_missing.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,主料,row,在库,在途,组合在库,组合在途,times
0,USABS10461-C,2,USABS10461-C,59.0,5.0,3.0,0.0,0.0,1
1,USABS11330-C,1,USABS11330-C,92.0,29.0,17.0,0.0,0.0,1
2,USABS12530-C,1,USABS12530-C,128.0,12.0,0.0,0.0,0.0,1
3,USABS13140-C,1,USABS13140-C,145.0,1.0,4.0,0.0,0.0,1
4,USABS1420-C,1,USABS1420-C,176.0,5.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
1631,USWSP4TH03B0-4-YC,1,USWSP4TH03B0-4-YC,268278.0,22.0,0.0,0.0,0.0,1
1632,USWSP54507B1-4-YC,1,USWSP54507B1-4-YC,268312.0,6.0,0.0,0.0,0.0,1
1633,USWSP54530B1-4-YC,1,USWSP54530B1-4-YC,268368.0,2.0,0.0,0.0,0.0,1
1634,USWSP65507B1-4-YC,1,USWSP65507B1-4-YC,268602.0,30.0,0.0,0.0,0.0,1


In [43]:
# 找出有库存信息的数据
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()
df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,59.0,2
1,92.0,1
2,128.0,1
3,145.0,1
4,176.0,1
...,...,...
1627,268278.0,1
1628,268312.0,1
1629,268368.0,1
1630,268602.0,1


In [44]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',                '主料',
                     'row',                '在库',                '在途',
                    '组合在库',              '组合在途',                '品类',
                      '层级',                '身份',             '品类负责人',
                    'Note'],
      dtype='object')
Index(['row', '数量'], dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,在库,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note,数量
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,195.0,50.0,14.0,0.0,BC,TOP1,老品,Hebe,,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,22.0,30.0,0.0,0.0,PSP,TOP10,老品,Hebe,,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,20.0,51.0,47.0,1.0,BC,TOP1,老品,Hebe,,0.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,73.0,40.0,4.0,0.0,BC,TOP1,老品,Hebe,,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,17.0,0.0,0.0,0.0,PSP,普通,老品,Hebe,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0
29609,,USSTW924647-C,,,,,,,,,...,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0
29610,,USSTW925504-C,,,,,,,,,...,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0
29611,,USSTW925507-C,,,,,,,,,...,,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0


In [45]:
# 找出没有库存信息的数据
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-4.无record缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU,数量
0,USCA23637-C1,1
1,USCA28198-C1,1
2,USCLK07233-C,5
3,USET03576-C1,1


In [46]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',                '主料',
                     'row',                '在库',                '在途',
                    '组合在库',              '组合在途',                '品类',
                      '层级',                '身份',             '品类负责人',
                    'Note',                '数量'],
      dtype='object')
Index(['SKU', '数量'], dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,在途,组合在库,组合在途,品类,层级,身份,品类负责人,Note,数量_1,数量_2
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,50.0,14.0,0.0,BC,TOP1,老品,Hebe,,0.0,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,30.0,0.0,0.0,PSP,TOP10,老品,Hebe,,0.0,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,51.0,47.0,1.0,BC,TOP1,老品,Hebe,,0.0,0.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,40.0,4.0,0.0,BC,TOP1,老品,Hebe,,0.0,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,0.0,0.0,0.0,PSP,普通,老品,Hebe,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0,0.0
29609,,USSTW924647-C,,,,,,,,,...,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0,0.0
29610,,USSTW925504-C,,,,,,,,,...,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0,0.0
29611,,USSTW925507-C,,,,,,,,,...,,,,STW,TOP10,新品,Hebe,"MRP only, not in last",0.0,0.0


In [47]:
df['无Record ID'] = [amount_1 + amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]
df['在途总数'] = [way_1 + way_2 for way_1, way_2 in zip(df['在途'].tolist(), df['组合在途'].tolist())]

list_column = df.columns.tolist()
list_column.remove('主料')
list_column.remove('在途')
list_column.remove('组合在途')
list_column.remove('品类负责人')
list_column.remove('数量_1')
list_column.remove('数量_2')
df = df[list_column]

df

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,Date_Left,row,在库,组合在库,品类,层级,身份,Note,无Record ID,在途总数
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,195,5568.0,195.0,14.0,BC,TOP1,老品,,0.0,50.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,18,38459.0,22.0,0.0,PSP,TOP10,老品,,0.0,30.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,20,5573.0,20.0,47.0,BC,TOP1,老品,,0.0,52.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,73,5366.0,73.0,4.0,BC,TOP1,老品,,0.0,40.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,17,38328.0,17.0,0.0,PSP,普通,老品,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,,STW,TOP10,新品,"MRP only, not in last",0.0,
29609,,USSTW924647-C,,,,,,,,,...,,,,,STW,TOP10,新品,"MRP only, not in last",0.0,
29610,,USSTW925504-C,,,,,,,,,...,,,,,STW,TOP10,新品,"MRP only, not in last",0.0,
29611,,USSTW925507-C,,,,,,,,,...,,,,,STW,TOP10,新品,"MRP only, not in last",0.0,


In [48]:
df_sale = pd.read_excel(price,
                        sheet_name=2,
                        header=0,
                        names=['SKU', '数量'],
                        usecols='D, F',
                        dtype=str).fillna('')

df_sale = df_sale[df_sale['SKU'] != ''].reset_index(drop=True)

df_sale['数量'] = df_sale['数量'].astype(int)
df_sale = df_sale.groupby(df_sale['SKU'], as_index=False).sum()

df_sale

,SKU,数量
0,USABS0L700-C,1
1,USABS11690-C,1
2,USABS12673-C,2
3,USABS13150-C,2
4,USABS1440-C,1
...,...,...
1983,USWS47500-C,1
1984,USWSP44302S0-4-YC,1
1985,USWSP44303R0-2-YC,1
1986,USWSP54722B1-4-YC,1


In [49]:
print(df_sale.columns)
print(df_price.columns)

df_sale = pd.merge(left=df_sale,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_sale

Index(['SKU', '数量'], dtype='object')
Index(['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途'], dtype='object')


,SKU,数量,主料,row,在库,在途,组合在库,组合在途
0,USABS0L700-C,1,USABS0L700-C,38.0,0.0,2.0,0.0,0.0
1,USABS11690-C,1,USABS11690-C,111.0,12.0,0.0,0.0,0.0
2,USABS12673-C,2,USABS12673-C,131.0,36.0,11.0,0.0,0.0
3,USABS13150-C,2,USABS13150-C,146.0,3.0,14.0,0.0,0.0
4,USABS1440-C,1,USABS1440-C,180.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1983,USWS47500-C,1,USWS47500-C,268099.0,23.0,0.0,0.0,0.0
1984,USWSP44302S0-4-YC,1,USWSP44302S0-4-YC,268222.0,5.0,0.0,0.0,0.0
1985,USWSP44303R0-2-YC,1,USWSP44303R0-2-YC,268225.0,1.0,0.0,0.0,0.0
1986,USWSP54722B1-4-YC,1,USWSP54722B1-4-YC,268440.0,5.0,1.0,0.0,0.0


In [50]:
df_temp = df_sale.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

,SKU,数量,主料,row,在库,在途,组合在库,组合在途,times
0,USABS0L700-C,1,USABS0L700-C,38.0,0.0,2.0,0.0,0.0,1
1,USABS11690-C,1,USABS11690-C,111.0,12.0,0.0,0.0,0.0,1
2,USABS12673-C,2,USABS12673-C,131.0,36.0,11.0,0.0,0.0,1
3,USABS13150-C,2,USABS13150-C,146.0,3.0,14.0,0.0,0.0,1
4,USABS1440-C,1,USABS1440-C,180.0,4.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...
1983,USWS47500-C,1,USWS47500-C,268099.0,23.0,0.0,0.0,0.0,1
1984,USWSP44302S0-4-YC,1,USWSP44302S0-4-YC,268222.0,5.0,0.0,0.0,0.0,1
1985,USWSP44303R0-2-YC,1,USWSP44303R0-2-YC,268225.0,1.0,0.0,0.0,0.0,1
1986,USWSP54722B1-4-YC,1,USWSP54722B1-4-YC,268440.0,5.0,1.0,0.0,0.0,1


In [51]:
# 找出有库存信息的数据
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()
df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

,row,数量
0,38.0,1
1,111.0,1
2,131.0,2
3,146.0,2
4,180.0,1
...,...,...
1978,268099.0,1
1979,268222.0,1
1980,268225.0,1
1981,268440.0,1


In [52]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',               'row',
                      '在库',              '组合在库',                '品类',
                      '层级',                '身份',              'Note',
              '无Record ID',              '在途总数'],
      dtype='object')
Index(['row', '数量'], dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,row,在库,组合在库,品类,层级,身份,Note,无Record ID,在途总数,数量
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,5568.0,195.0,14.0,BC,TOP1,老品,,0.0,50.0,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,38459.0,22.0,0.0,PSP,TOP10,老品,,0.0,30.0,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,5573.0,20.0,47.0,BC,TOP1,老品,,0.0,52.0,0.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,5366.0,73.0,4.0,BC,TOP1,老品,,0.0,40.0,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,38328.0,17.0,0.0,PSP,普通,老品,,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29609,,USSTW924647-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29610,,USSTW925504-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29611,,USSTW925507-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0


In [53]:
# 找出没有库存信息的数据
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-5.sale缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

,SKU,数量
0,USACC29209-C1,1
1,USACH11879J-C,1
2,USBMR32438-C,1
3,USCA28198-C1,1
4,USFSU149A0-C,1


In [54]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

Index([               '序号',               'SKU',               '替代料',
                   'YKD库存', 2024-04-21 00:00:01, 2024-04-19 00:00:01,
       2024-04-21 00:00:02, 2024-04-20 00:00:02, 2024-04-19 00:00:02,
       2024-04-18 00:00:02, 2024-04-17 00:00:02, 2024-04-16 00:00:02,
       2024-04-15 00:00:02,       'weekly_sale',              '改价记录',
                    '断货时间',           '总品类调价记录',            '每日调价记录',
                    '上次序号',         'Date_Left',               'row',
                      '在库',              '组合在库',                '品类',
                      '层级',                '身份',              'Note',
              '无Record ID',              '在途总数',                '数量'],
      dtype='object')
Index(['SKU', '数量'], dtype='object')


,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,在库,组合在库,品类,层级,身份,Note,无Record ID,在途总数,数量_1,数量_2
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,195.0,14.0,BC,TOP1,老品,,0.0,50.0,0.0,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,22.0,0.0,PSP,TOP10,老品,,0.0,30.0,0.0,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,20.0,47.0,BC,TOP1,老品,,0.0,52.0,0.0,0.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,73.0,4.0,BC,TOP1,老品,,0.0,40.0,0.0,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,17.0,0.0,PSP,普通,老品,,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0,0.0
29609,,USSTW924647-C,,,,,,,,,...,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0,0.0
29610,,USSTW925504-C,,,,,,,,,...,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0,0.0
29611,,USSTW925507-C,,,,,,,,,...,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0,0.0


In [55]:
df['Date_Right'] = [amount_1 + amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]

list_column = df.columns.tolist()
list_column.remove('row')
list_column.remove('数量_1')
list_column.remove('数量_2')
df = df[list_column]

df

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,Date_Left,在库,组合在库,品类,层级,身份,Note,无Record ID,在途总数,Date_Right
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,195,195.0,14.0,BC,TOP1,老品,,0.0,50.0,0.0
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,18,22.0,0.0,PSP,TOP10,老品,,0.0,30.0,0.0
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,20,20.0,47.0,BC,TOP1,老品,,0.0,52.0,0.0
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,73,73.0,4.0,BC,TOP1,老品,,0.0,40.0,0.0
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,17,17.0,0.0,PSP,普通,老品,,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29609,,USSTW924647-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29610,,USSTW925504-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0
29611,,USSTW925507-C,,,,,,,,,...,,,,STW,TOP10,新品,"MRP only, not in last",0.0,,0.0


In [56]:
df['改价'] = ['=IFERROR(IF(J' + str(i+3) + '-$M' + str(i+3) + '>0,"海运上架",IF(J' + str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['实时库存'] = ['=L' + str(i+3) + '-I' + str(i+3) for i in range(len(df))]
df['weekly_sale'] = ['=SUM(P' + str(i+3) + ':W' + str(i+3) + ')' for i in range(len(df))]
df['改价+断货'] = ['=TEXTJOIN(";",1,Y' + str(i+3) + ',Z' + str(i+3) + ')' for i in range(len(df))]
df['总库存'] = ['=J' + str(i+3) + '+K' + str(i+3) for i in range(len(df))]
df['总品类调价记录_2'] = ['=TEXTJOIN(";",TRUE,AC' + str(i+3) + ':AD' + str(i+3) + ')' for i in range(len(df))]
df['合计改价记录'] = ['=TEXTJOIN(";",1,AE' + str(i+3) + ',Y' + str(i+3) + ')' for i in range(len(df))]

df

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_94909/1234657324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['改价'] = ['=IFERROR(IF(J' + str(i+3) + '-$M' + str(i+3) + '>0,"海运上架",IF(J' + str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_94909/1234657324.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['实时库存'] = ['=L' + str(i+3) + '-I' + str(i+3) for i in range(len(df))]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T

,序号,SKU,替代料,YKD库存,2024-04-21 00:00:01,2024-04-19 00:00:01,2024-04-21 00:00:02,2024-04-20 00:00:02,2024-04-19 00:00:02,2024-04-18 00:00:02,...,Note,无Record ID,在途总数,Date_Right,改价,实时库存,改价+断货,总库存,总品类调价记录_2,合计改价记录
0,1,USBCB72928BLR-C,USBCB72928BLR,195,196,202,1,3,3,0,...,,0.0,50.0,0.0,"=IFERROR(IF(J3-$M3>0,""海运上架"",IF(J3<=0,""断货"",""""))...",=L3-I3,"=TEXTJOIN("";"",1,Y3,Z3)",=J3+K3,"=TEXTJOIN("";"",TRUE,AC3:AD3)","=TEXTJOIN("";"",1,AE3,Y3)"
1,2,USPSP14045D-C,USPSP14045D,18,17,17,0,0,0,0,...,,0.0,30.0,0.0,"=IFERROR(IF(J4-$M4>0,""海运上架"",IF(J4<=0,""断货"",""""))...",=L4-I4,"=TEXTJOIN("";"",1,Y4,Z4)",=J4+K4,"=TEXTJOIN("";"",TRUE,AC4:AD4)","=TEXTJOIN("";"",1,AE4,Y4)"
2,3,USBCB75352ZLR-C,USBCB75352ZLR,20,22,23,1,0,0,0,...,,0.0,52.0,0.0,"=IFERROR(IF(J5-$M5>0,""海运上架"",IF(J5<=0,""断货"",""""))...",=L5-I5,"=TEXTJOIN("";"",1,Y5,Z5)",=J5+K5,"=TEXTJOIN("";"",TRUE,AC5:AD5)","=TEXTJOIN("";"",1,AE5,Y5)"
3,4,USBCB08180ZLR-C,USBCB08180ZLR,73,73,73,0,0,0,0,...,,0.0,40.0,0.0,"=IFERROR(IF(J6-$M6>0,""海运上架"",IF(J6<=0,""断货"",""""))...",=L6-I6,"=TEXTJOIN("";"",1,Y6,Z6)",=J6+K6,"=TEXTJOIN("";"",TRUE,AC6:AD6)","=TEXTJOIN("";"",1,AE6,Y6)"
4,5,USPSP01008D-C,USPSP01008D,17,17,17,0,0,0,0,...,,0.0,0.0,0.0,"=IFERROR(IF(J7-$M7>0,""海运上架"",IF(J7<=0,""断货"",""""))...",=L7-I7,"=TEXTJOIN("";"",1,Y7,Z7)",=J7+K7,"=TEXTJOIN("";"",TRUE,AC7:AD7)","=TEXTJOIN("";"",1,AE7,Y7)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,,USSTW924638-C,,,,,,,,,...,"MRP only, not in last",0.0,,0.0,"=IFERROR(IF(J29611-$M29611>0,""海运上架"",IF(J29611<...",=L29611-I29611,"=TEXTJOIN("";"",1,Y29611,Z29611)",=J29611+K29611,"=TEXTJOIN("";"",TRUE,AC29611:AD29611)","=TEXTJOIN("";"",1,AE29611,Y29611)"
29609,,USSTW924647-C,,,,,,,,,...,"MRP only, not in last",0.0,,0.0,"=IFERROR(IF(J29612-$M29612>0,""海运上架"",IF(J29612<...",=L29612-I29612,"=TEXTJOIN("";"",1,Y29612,Z29612)",=J29612+K29612,"=TEXTJOIN("";"",TRUE,AC29612:AD29612)","=TEXTJOIN("";"",1,AE29612,Y29612)"
29610,,USSTW925504-C,,,,,,,,,...,"MRP only, not in last",0.0,,0.0,"=IFERROR(IF(J29613-$M29613>0,""海运上架"",IF(J29613<...",=L29613-I29613,"=TEXTJOIN("";"",1,Y29613,Z29613)",=J29613+K29613,"=TEXTJOIN("";"",TRUE,AC29613:AD29613)","=TEXTJOIN("";"",1,AE29613,Y29613)"
29611,,USSTW925507-C,,,,,,,,,...,"MRP only, not in last",0.0,,0.0,"=IFERROR(IF(J29614-$M29614>0,""海运上架"",IF(J29614<...",=L29614-I29614,"=TEXTJOIN("";"",1,Y29614,Z29614)",=J29614+K29614,"=TEXTJOIN("";"",TRUE,AC29614:AD29614)","=TEXTJOIN("";"",1,AE29614,Y29614)"


In [57]:
list_column = df.columns.tolist()
df = df[['序号', '层级', 'SKU', '替代料', '品类', '身份', '在途总数', '改价', '无Record ID', '实时库存', '组合在库', '在库', 'Date_Left'] +
         list_column[4:6] +
         ['Date_Right'] + 
         list_column[6:13] +
         ['weekly_sale', '改价记录', '断货时间', '改价+断货', '总库存', '总品类调价记录', '每日调价记录', '总品类调价记录_2', '合计改价记录', 'Note', '上次序号']]
list_column = df.columns.tolist()
list_column[list_column.index('组合在库')] = '组合料库存'
list_column[list_column.index('在库')] = 'YKD库存'
df.columns = list_column

df.sort_values(by=['上次序号', 'SKU'], ascending=[True, True]).reset_index(drop=True)
df['序号'] = [i+1 for i in range(len(df))]

df

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_94909/3288800670.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['序号'] = [i+1 for i in range(len(df))]


,序号,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,...,改价记录,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,Note,上次序号
0,1,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,50.0,"=IFERROR(IF(J3-$M3>0,""海运上架"",IF(J3<=0,""断货"",""""))...",0.0,=L3-I3,...,,,"=TEXTJOIN("";"",1,Y3,Z3)",=J3+K3,,,"=TEXTJOIN("";"",TRUE,AC3:AD3)","=TEXTJOIN("";"",1,AE3,Y3)",,1
1,2,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,30.0,"=IFERROR(IF(J4-$M4>0,""海运上架"",IF(J4<=0,""断货"",""""))...",0.0,=L4-I4,...,,,"=TEXTJOIN("";"",1,Y4,Z4)",=J4+K4,,,"=TEXTJOIN("";"",TRUE,AC4:AD4)","=TEXTJOIN("";"",1,AE4,Y4)",,2
2,3,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,老品,52.0,"=IFERROR(IF(J5-$M5>0,""海运上架"",IF(J5<=0,""断货"",""""))...",0.0,=L5-I5,...,,,"=TEXTJOIN("";"",1,Y5,Z5)",=J5+K5,,,"=TEXTJOIN("";"",TRUE,AC5:AD5)","=TEXTJOIN("";"",1,AE5,Y5)",,3
3,4,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,40.0,"=IFERROR(IF(J6-$M6>0,""海运上架"",IF(J6<=0,""断货"",""""))...",0.0,=L6-I6,...,,,"=TEXTJOIN("";"",1,Y6,Z6)",=J6+K6,,,"=TEXTJOIN("";"",TRUE,AC6:AD6)","=TEXTJOIN("";"",1,AE6,Y6)",,4
4,5,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,"=IFERROR(IF(J7-$M7>0,""海运上架"",IF(J7<=0,""断货"",""""))...",0.0,=L7-I7,...,,,"=TEXTJOIN("";"",1,Y7,Z7)",=J7+K7,,,"=TEXTJOIN("";"",TRUE,AC7:AD7)","=TEXTJOIN("";"",1,AE7,Y7)",,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,29609,TOP10,USSTW924638-C,,STW,新品,,"=IFERROR(IF(J29611-$M29611>0,""海运上架"",IF(J29611<...",0.0,=L29611-I29611,...,,,"=TEXTJOIN("";"",1,Y29611,Z29611)",=J29611+K29611,,,"=TEXTJOIN("";"",TRUE,AC29611:AD29611)","=TEXTJOIN("";"",1,AE29611,Y29611)","MRP only, not in last",
29609,29610,TOP10,USSTW924647-C,,STW,新品,,"=IFERROR(IF(J29612-$M29612>0,""海运上架"",IF(J29612<...",0.0,=L29612-I29612,...,,,"=TEXTJOIN("";"",1,Y29612,Z29612)",=J29612+K29612,,,"=TEXTJOIN("";"",TRUE,AC29612:AD29612)","=TEXTJOIN("";"",1,AE29612,Y29612)","MRP only, not in last",
29610,29611,TOP10,USSTW925504-C,,STW,新品,,"=IFERROR(IF(J29613-$M29613>0,""海运上架"",IF(J29613<...",0.0,=L29613-I29613,...,,,"=TEXTJOIN("";"",1,Y29613,Z29613)",=J29613+K29613,,,"=TEXTJOIN("";"",TRUE,AC29613:AD29613)","=TEXTJOIN("";"",1,AE29613,Y29613)","MRP only, not in last",
29611,29612,TOP10,USSTW925507-C,,STW,新品,,"=IFERROR(IF(J29614-$M29614>0,""海运上架"",IF(J29614<...",0.0,=L29614-I29614,...,,,"=TEXTJOIN("";"",1,Y29614,Z29614)",=J29614+K29614,,,"=TEXTJOIN("";"",TRUE,AC29614:AD29614)","=TEXTJOIN("";"",1,AE29614,Y29614)","MRP only, not in last",


In [58]:
for i in tqdm(range(len(df)), desc='Progress', ncols=77):
    try:
        df.loc[i, '在途总数'] = int(df.loc[i, '在途总数'].strip())
    except:
        pass
    
    try:
        df.loc[i, '无Record ID'] = int(df.loc[i, '无Record ID'].strip())
    except:
        pass
    
    try:
        df.loc[i, '组合料库存'] = int(df.loc[i, '组合料库存'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'YKD库存'] = int(df.loc[i, 'YKD库存'].strip())
    except:
        pass
    
    try:
        df.iloc[i, 12] = int(df.iloc[i, 12].strip())
    except:
        pass
    
    try:
        df.iloc[i, 13] = int(df.iloc[i, 13].strip())
    except:
        pass
    
    try:
        df.iloc[i, 14] = int(df.iloc[i, 14].strip())
    except:
        pass
    
    try:
        df.iloc[i, 15] = int(df.iloc[i, 15].strip())
    except:
        pass
    
    try:
        df.iloc[i, 16] = int(df.iloc[i, 16].strip())
    except:
        pass
    
    try:
        df.iloc[i, 17] = int(df.iloc[i, 17].strip())
    except:
        pass
    
    try:
        df.iloc[i, 18] = int(df.iloc[i, 18].strip())
    except:
        pass
    
    try:
        df.iloc[i, 19] = int(df.iloc[i, 19].strip())
    except:
        pass
    
    try:
        df.iloc[i, 20] = int(df.iloc[i, 20].strip())
    except:
        pass
    
    try:
        df.iloc[i, 21] = int(df.iloc[i, 21].strip())
    except:
        pass
    
    try:
        df.iloc[i, 22] = int(df.iloc[i, 22].strip())
    except:
        pass
    
    try:
        df.iloc[i, 23] = int(df.iloc[i, 23].strip())
    except:
        pass
    
df.to_excel('./output/am-colorless.xlsx', index=False, startrow=1)

df

Progress: 100%|██████████████████████| 29613/29613 [00:18<00:00, 1562.12it/s]


,序号,层级,SKU,替代料,品类,身份,在途总数,改价,无Record ID,实时库存,...,改价记录,断货时间,改价+断货,总库存,总品类调价记录,每日调价记录,总品类调价记录_2,合计改价记录,Note,上次序号
0,1,TOP1,USBCB72928BLR-C,USBCB72928BLR,BC,老品,50.0,"=IFERROR(IF(J3-$M3>0,""海运上架"",IF(J3<=0,""断货"",""""))...",0.0,=L3-I3,...,,,"=TEXTJOIN("";"",1,Y3,Z3)",=J3+K3,,,"=TEXTJOIN("";"",TRUE,AC3:AD3)","=TEXTJOIN("";"",1,AE3,Y3)",,1
1,2,TOP10,USPSP14045D-C,USPSP14045D,PSP,老品,30.0,"=IFERROR(IF(J4-$M4>0,""海运上架"",IF(J4<=0,""断货"",""""))...",0.0,=L4-I4,...,,,"=TEXTJOIN("";"",1,Y4,Z4)",=J4+K4,,,"=TEXTJOIN("";"",TRUE,AC4:AD4)","=TEXTJOIN("";"",1,AE4,Y4)",,2
2,3,TOP1,USBCB75352ZLR-C,USBCB75352ZLR,BC,老品,52.0,"=IFERROR(IF(J5-$M5>0,""海运上架"",IF(J5<=0,""断货"",""""))...",0.0,=L5-I5,...,,,"=TEXTJOIN("";"",1,Y5,Z5)",=J5+K5,,,"=TEXTJOIN("";"",TRUE,AC5:AD5)","=TEXTJOIN("";"",1,AE5,Y5)",,3
3,4,TOP1,USBCB08180ZLR-C,USBCB08180ZLR,BC,老品,40.0,"=IFERROR(IF(J6-$M6>0,""海运上架"",IF(J6<=0,""断货"",""""))...",0.0,=L6-I6,...,,,"=TEXTJOIN("";"",1,Y6,Z6)",=J6+K6,,,"=TEXTJOIN("";"",TRUE,AC6:AD6)","=TEXTJOIN("";"",1,AE6,Y6)",,4
4,5,普通,USPSP01008D-C,USPSP01008D,PSP,老品,0.0,"=IFERROR(IF(J7-$M7>0,""海运上架"",IF(J7<=0,""断货"",""""))...",0.0,=L7-I7,...,,,"=TEXTJOIN("";"",1,Y7,Z7)",=J7+K7,,,"=TEXTJOIN("";"",TRUE,AC7:AD7)","=TEXTJOIN("";"",1,AE7,Y7)",,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29608,29609,TOP10,USSTW924638-C,,STW,新品,,"=IFERROR(IF(J29611-$M29611>0,""海运上架"",IF(J29611<...",0.0,=L29611-I29611,...,,,"=TEXTJOIN("";"",1,Y29611,Z29611)",=J29611+K29611,,,"=TEXTJOIN("";"",TRUE,AC29611:AD29611)","=TEXTJOIN("";"",1,AE29611,Y29611)","MRP only, not in last",
29609,29610,TOP10,USSTW924647-C,,STW,新品,,"=IFERROR(IF(J29612-$M29612>0,""海运上架"",IF(J29612<...",0.0,=L29612-I29612,...,,,"=TEXTJOIN("";"",1,Y29612,Z29612)",=J29612+K29612,,,"=TEXTJOIN("";"",TRUE,AC29612:AD29612)","=TEXTJOIN("";"",1,AE29612,Y29612)","MRP only, not in last",
29610,29611,TOP10,USSTW925504-C,,STW,新品,,"=IFERROR(IF(J29613-$M29613>0,""海运上架"",IF(J29613<...",0.0,=L29613-I29613,...,,,"=TEXTJOIN("";"",1,Y29613,Z29613)",=J29613+K29613,,,"=TEXTJOIN("";"",TRUE,AC29613:AD29613)","=TEXTJOIN("";"",1,AE29613,Y29613)","MRP only, not in last",
29611,29612,TOP10,USSTW925507-C,,STW,新品,,"=IFERROR(IF(J29614-$M29614>0,""海运上架"",IF(J29614<...",0.0,=L29614-I29614,...,,,"=TEXTJOIN("";"",1,Y29614,Z29614)",=J29614+K29614,,,"=TEXTJOIN("";"",TRUE,AC29614:AD29614)","=TEXTJOIN("";"",1,AE29614,Y29614)","MRP only, not in last",


# 6. 格式更改

In [59]:
from openpyxl import load_workbook

In [60]:
wb_this = load_workbook('./output/am-colorless.xlsx')
ws_this = wb_this.worksheets[0]

In [61]:
coef = 1.15
ws_this.column_dimensions['A'].width = coef * 6
ws_this.column_dimensions['B'].width = coef * 9
ws_this.column_dimensions['C'].width = coef * 15
ws_this.column_dimensions['D'].width = coef * 15
ws_this.column_dimensions['E'].width = coef * 9
ws_this.column_dimensions['F'].width = coef * 9
ws_this.column_dimensions['G'].width = coef * 7
ws_this.column_dimensions['H'].width = coef * 7
ws_this.column_dimensions['I'].width = coef * 10
ws_this.column_dimensions['J'].width = coef * 10
ws_this.column_dimensions['K'].width = coef * 10
ws_this.column_dimensions['L'].width = coef * 8
ws_this.column_dimensions['M'].width = coef * 10
ws_this.column_dimensions['N'].width = coef * 10
ws_this.column_dimensions['O'].width = coef * 10
ws_this.column_dimensions['P'].width = coef * 10
ws_this.column_dimensions['Q'].width = coef * 10
ws_this.column_dimensions['R'].width = coef * 10
ws_this.column_dimensions['S'].width = coef * 10
ws_this.column_dimensions['T'].width = coef * 10
ws_this.column_dimensions['U'].width = coef * 10
ws_this.column_dimensions['V'].width = coef * 10
ws_this.column_dimensions['W'].width = coef * 10
ws_this.column_dimensions['X'].width = coef * 8
ws_this.column_dimensions['Y'].width = coef * 10
ws_this.column_dimensions['Z'].width = coef * 10
ws_this.column_dimensions['AA'].width = coef * 10
ws_this.column_dimensions['AB'].width = coef * 10
ws_this.column_dimensions['AC'].width = coef * 10
ws_this.column_dimensions['AD'].width = coef * 10
ws_this.column_dimensions['AE'].width = coef * 10
ws_this.column_dimensions['AF'].width = coef * 10
ws_this.column_dimensions['AG'].width = coef * 10
ws_this.column_dimensions['AH'].width = coef * 6

ws_this.row_dimensions[1].height = 25
ws_this.row_dimensions[2].height = 34
for i in range(3, ws_this.max_row+1):
    ws_this.row_dimensions[i].height = 16.5

In [62]:
from openpyxl.styles import Font, PatternFill, Alignment

In [63]:
ws_this['A1'].value = str(datetime.now().month) + '/' + str(datetime.now().day)
ws_this['A1'].font = Font(b=True)
ws_this['A1'].number_format = 'yyyy/m/d;@'

ws_this['H1'].value = '任务栏'
ws_this['H1'].font = Font(bold=True)

ws_this['P1'].value = '近8天 周销情况'
ws_this.merge_cells('P1:W1')

ws_this['Y1'].value = '10.5-J310.18-T6(日期最新在后面）'

In [64]:
for list_cell in ws_this['A2:K2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
ws_this['L2'].fill = PatternFill(patternType='solid', fgColor='FF92D050')
        
for list_cell in ws_this['M2:O2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FF92D050')
    
for list_cell in ws_this['P2:W2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FFFABF8E')

for list_cell in ws_this['X2:AA2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
ws_this['AB2'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

for list_cell in ws_this['AC2:AF2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

In [65]:
wb_this.save('./断货改价-大账号-上午 ' + str(datetime.now().month) + '.' + str(datetime.now().day) + '.xlsx')
wb_this.close()

print('Done ~')

Done ~
